In [1]:
#### Convert coffea output into root TH2's to feed into TUnfold
import coffea
import pickle
import uproot
import awkward as ak
import matplotlib.pyplot as plt
import numpy as np
import os
# import ROOT

In [2]:
from python.plugins import addFiles
# files = ["coffeaOutput/trijetHists_JetHT_pt200.0_rappidity2.5_bbloosejesjec2018.pkl", "coffeaOutput/trijetHists_JetHT_pt200.0_rappidity2.5_bbloosejesjec2017.pkl", "coffeaOutput/trijetHists_JetHT_pt200.0_rappidity2.5_bbloosejesjec2016.pkl"]
#files = ["coffeaOutput/trijetHists_wXSscaling_QCDsim_pt200.0rapidity2.5_bbloosejesjec2018.pkl", "coffeaOutput/trijetHists_wXSscaling_QCDsim_pt200.0rapidity2.5_bbloosejesjec2017.pkl", "coffeaOutput/trijetHists_wXSscaling_QCDsim_pt200.0rapidity2.5_bbloosejesjec2016.pkl"]
# files = ["coffeaOutput/dijetHists_JetHT_pt200.0_rapidity2.5jesjec2018.pkl", "coffeaOutput/dijetHists_JetHT_pt200.0_rapidity2.5jesjec2016.pkl", "coffeaOutput/dijetHists_JetHT_pt200.0_rapidity2.5jesjec2017.pkl"]
# files = ["coffeaOutput/dijetHists_wXSscaling_QCDsim_pt200.0_rapidity2.5jesjec2017.pkl", "coffeaOutput/dijetHists_wXSscaling_QCDsim_pt200.0_rapidity2.5jesjec2018.pkl", "coffeaOutput/dijetHists_wXSscaling_QCDsim_pt200.0_rapidity2.5jesjec2016.pkl"]
# files = ["coffeaOutput/trijetHists_wXSscaling_QCDsim_pt200.0rapidity2.5_bloosejesjec2016.pkl", "coffeaOutput/trijetHists_wXSscaling_QCDsim_pt200.0rapidity2.5_bloosejesjec2017.pkl", "coffeaOutput/trijetHists_wXSscaling_QCDsim_pt200.0rapidity2.5_bloosejesjec2018.pkl"]
files = ["coffeaOutput/dijetHists_wXSscaling_QCDsim_pt200.0_rapidity2.5jesjecL1PU2016.pkl", "coffeaOutput/dijetHistsTest_wXSscaling_QCDsim_pt200.0_rapidity2.5jesjecL1PU2017.pkl", "coffeaOutput/dijetHistsTest_wXSscaling_QCDsim_pt200.0_rapidity2.5jesjecL1PU2018.pkl"]
# combinedFile = addFiles(files)
# print(combinedFile)




In [3]:
def combineSystFiles(prepstr, IOV):
    unc_srcs = ["nominal", "jer", "AbsoluteMPFBias","AbsoluteScale","AbsoluteStat","FlavorQCD","Fragmentation","PileUpDataMC","PileUpPtBB","PileUpPtEC1","PileUpPtEC2","PileUpPtHF",
"PileUpPtRef","RelativeFSR","RelativeJEREC1","RelativeJEREC2","RelativeJERHF","RelativePtBB","RelativePtEC1","RelativePtEC2","RelativePtHF","RelativeBal","RelativeSample","RelativeStatEC", "RelativeStatFSR","RelativeStatHF","SinglePionECAL","SinglePionHCAL","TimePtEta"]
    files = []
    for src in unc_srcs:
        files.append(prepstr+src+IOV+".pkl")
    print("Final list of files ", files)
    combined = addFiles(files)
    outputFilename = prepstr+"allUnc"+IOV+".pkl"
    with open(outputFilename, "wb") as f:
        pickle.dump( combined, f)
    return combined

In [4]:
combineSystFiles("coffeaOutput/dijet/dijetHistsTest_wXSscaling_QCDsim_pt200.0_rapidity2.5_", "2018")

Final list of files  ['coffeaOutput/dijet/dijetHistsTest_wXSscaling_QCDsim_pt200.0_rapidity2.5_nominal2018.pkl', 'coffeaOutput/dijet/dijetHistsTest_wXSscaling_QCDsim_pt200.0_rapidity2.5_jer2018.pkl', 'coffeaOutput/dijet/dijetHistsTest_wXSscaling_QCDsim_pt200.0_rapidity2.5_AbsoluteMPFBias2018.pkl', 'coffeaOutput/dijet/dijetHistsTest_wXSscaling_QCDsim_pt200.0_rapidity2.5_AbsoluteScale2018.pkl', 'coffeaOutput/dijet/dijetHistsTest_wXSscaling_QCDsim_pt200.0_rapidity2.5_AbsoluteStat2018.pkl', 'coffeaOutput/dijet/dijetHistsTest_wXSscaling_QCDsim_pt200.0_rapidity2.5_FlavorQCD2018.pkl', 'coffeaOutput/dijet/dijetHistsTest_wXSscaling_QCDsim_pt200.0_rapidity2.5_Fragmentation2018.pkl', 'coffeaOutput/dijet/dijetHistsTest_wXSscaling_QCDsim_pt200.0_rapidity2.5_PileUpDataMC2018.pkl', 'coffeaOutput/dijet/dijetHistsTest_wXSscaling_QCDsim_pt200.0_rapidity2.5_PileUpPtBB2018.pkl', 'coffeaOutput/dijet/dijetHistsTest_wXSscaling_QCDsim_pt200.0_rapidity2.5_PileUpPtEC12018.pkl', 'coffeaOutput/dijet/dijetHistsTes

{'jet_mass': Hist(
   StrCategory(['jet1', 'jet2'], growth=True, name='jetNumb', label='Jet'),
   StrCategory(['Gluon', 'UDS', 'Charm', 'Bottom'], growth=True, name='partonFlav', label='Parton Flavour'),
   Variable(array([0.00e+00, 5.00e-01, 1.00e+00, 3.00e+00, 5.00e+00, 7.50e+00,
        1.00e+01, 1.50e+01, 2.00e+01, 3.00e+01, 4.00e+01, 5.00e+01,
        6.00e+01, 7.00e+01, 8.00e+01, 9.00e+01, 1.00e+02, 1.25e+02,
        1.50e+02, 1.75e+02, 2.00e+02, 2.25e+02, 2.50e+02, 6.25e+02,
        1.00e+03]), name='mreco', label='m_{RECO} (GeV)'),
   storage=Weight()) # Sum: WeightedSum(value=1.80334e+06, variance=1.80334e+06) (WeightedSum(value=1.80578e+06, variance=1.80578e+06) with flow),
 'jet_pt': Hist(
   StrCategory(['jet1', 'jet2'], growth=True, name='jetNumb', label='Jet'),
   StrCategory(['Gluon', 'UDS', 'Charm', 'Bottom'], growth=True, name='partonFlav', label='Parton Flavour'),
   Variable([200, 280, 360, 450, 520, 630, 690, 750, 800, 1300, 13000], name='ptreco', label='p_{T,RECO} 

In [5]:
fname = "coffeaOutput/trijetHists_wXSscaling_QCDsim_pt200.0rapidity2.5_bloosejesjecALL.pkl"
with open(fname, "rb") as f:
    result = pickle.load( f )
print("values: ", result['response_matrix_u'].project("ptgen", "mgen").values())
print("variances: ", result['response_matrix_u'].project("ptgen", "mgen").variances())

values:  [[0.00000000e+00 1.64840196e-01 1.18401463e+02 7.38814285e+03
  1.20690132e+05 1.64512215e+05 7.51661600e+04 2.98801216e+04
  9.88462497e+03 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 4.83658556e-01 6.37905239e+00 4.69203085e+02
  1.04213764e+04 2.27263886e+04 1.60053940e+04 7.47822871e+03
  7.56164125e+03 1.34039913e+02 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 2.38251690e+00 5.47565308e+01
  1.01231259e+03 3.37410983e+03 2.72137159e+03 1.88611583e+03
  1.91757304e+03 4.36158293e+02 9.99400201e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 4.05365298e-03 5.08970139e+00
  9.72279846e+01 3.71238657e+02 3.95851684e+02 2.78597575e+02
  3.30672076e+02 1.42670520e+02 1.83631184e+01 1.38024987e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 1.30384677e-01
  2.02193621e+01 1.02960264e+02 1.35803815e+02 9.55402662e+01
  1.18882460e+02 4.99535960e+01 2.74412044e+01 5.02359511e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 1.2651172

In [6]:
%matplotlib inline
# fname = "coffeaOutput/dijetHists_wXSscaling_QCDsim_pt200.0_rapidity2.5jesjecALL.pkl"
fname = "coffeaOutput/trijetHists_wXSscaling_QCDsim_pt200.0rapidity2.5_bloosejesjecALL.pkl"
# fname = "coffeaOutput/trijetHists_JetHT_pt200.0_rappidity2.5_Nonejesjec2016APV.pkl"
# fname ="coffeaOutput/trijetHistsTest_JetHT_pt200.0_eta2.4_bbloose.pkl"
eras = ["2016", "2017", "2018", "2016APV"]
dir = "rootFiles/"
for year in eras:
    print(year)
    if year in fname:
        print(year)
        year_str = year
        break
    else:
        year_str = "ALL"
with open(fname, "rb") as f:
    result = pickle.load( f )
axis_names = [ax for ax in result['jet_pt_mass_reco_u'].project("dataset").axes[0]]
print(axis_names)
cats = [cat for cat in result['jet_pt_mass_reco_u'][{'ptreco':sum, 'dataset':sum, 'mreco':sum}].axes[0]]
print(cats)
if "JetHT" in fname:
    if "dijet" in  fname:
        rootfname = 'dijetHistsJetHT_jec_' + year_str + '.root'
    elif "trijet" in fname:
        rootfname = 'trijetHistsJetHT_jec_' + year_str + '.root'
elif "QCD" in fname:
    if "dijet" in  fname:
        rootfname = 'dijetHistsQCDsim_jec_' + year_str + '.root'
    # integrate and sum over axes
    elif "trijet" in fname:
        rootfname = 'trijetHistsQCDsim_jec' + year_str + '.root'
if os.path.exists(dir + rootfname):
    rootfile = uproot.recreate(dir+rootfname)
else:
    rootfile = uproot.create(dir+rootfname)
    #### only add hists containing gen if SIM files
response_entries = {}
if "QCD" in fname:
    for syst in cats:
        rootfile['ptgen_mgen_u_'+syst] = result['response_matrix_u'][{'syst':syst}].project("ptgen", "mgen")
        rootfile['ptgen_mgen_g_'+syst] = result['response_matrix_g'][{'syst':syst}].project("ptgen","mgen")
        rootfile['ptreco_mreco_u_'+syst] = result['response_matrix_u'][{'syst':syst}].project("ptreco", "mreco")
        rootfile['ptreco_mreco_g_'+syst] = result['response_matrix_g'][{'syst':syst}].project("ptreco","mreco")
        rootfile['misses_ptgen_mgen_'+syst] = result['misses'][{'syst':syst}].project("ptgen", "mgen")
        rootfile['fakes_ptreco_mreco_'+syst] = result['fakes'][{'syst':syst}].project("ptreco","mreco")
        response_matrix_u_values, ptreco_edges, mreco_edges, ptgen_edges, mgen_edges = result['response_matrix_u'][{'syst':syst}].project("ptreco", "mreco", "ptgen", "mgen").to_numpy(flow=True)
        response_matrix_g_values, ptreco_edges, mreco_edges, ptgen_edges, mgen_edges= result['response_matrix_g'][{'syst':syst}].project("ptreco", "mreco", "ptgen", "mgen").to_numpy(flow=True)
        print(response_matrix_g_values.shape)
        ptreco_centers = (ptreco_edges[:-1]+ptreco_edges[1:])/2
        print(ptreco_centers[np.newaxis])
        mreco_centers = (mreco_edges[:-1]+mreco_edges[1:])/2
        ptgen_centers = (ptgen_edges[:-1]+ptgen_edges[1:])/2
        mgen_centers = (mgen_edges[:-1]+mgen_edges[1:])/2
        print(np.shape(response_matrix_g_values))
        print(np.shape(response_matrix_g_values[np.newaxis]))
        response_entries.update({'groomed_'+syst:response_matrix_g_values[np.newaxis],
                           'ungroomed_'+syst:response_matrix_u_values[np.newaxis]})
    rootfile['response']=response_entries
else:
    for syst in cats:
        rootfile['ptreco_mreco_u_'+syst] = result['jet_pt_mass_reco_u'][{'syst':syst}].project("ptreco", "mreco")
        rootfile['ptreco_mreco_g_'+syst] = result['jet_pt_mass_reco_g'][{'syst':syst}].project("ptreco","mreco")
rootfile.close()

2016
2017
2018
2016APV
['/QCD_Pt_1000to1400_TuneCP5_13TeV_pythia8/RunIISummer20UL16NanoAODv9-106X_mcRun2_asymptotic_v17-v1/NANOAODSIM', '/QCD_Pt_170to300_TuneCP5_13TeV_pythia8/RunIISummer20UL16NanoAODv9-106X_mcRun2_asymptotic_v17-v1/NANOAODSIM', '/QCD_Pt_1400to1800_TuneCP5_13TeV_pythia8/RunIISummer20UL16NanoAODv9-106X_mcRun2_asymptotic_v17-v1/NANOAODSIM', '/QCD_Pt_600to800_TuneCP5_13TeV_pythia8/RunIISummer20UL16NanoAODv9-106X_mcRun2_asymptotic_v17-v1/NANOAODSIM', '/QCD_Pt_800to1000_TuneCP5_13TeV_pythia8/RunIISummer20UL16NanoAODv9-106X_mcRun2_asymptotic_v17-v1/NANOAODSIM', '/QCD_Pt_470to600_TuneCP5_13TeV_pythia8/RunIISummer20UL16NanoAODv9-106X_mcRun2_asymptotic_v17-v1/NANOAODSIM', '/QCD_Pt_300to470_TuneCP5_13TeV_pythia8/RunIISummer20UL16NanoAODv9-106X_mcRun2_asymptotic_v17-v1/NANOAODSIM', '/QCD_Pt_3200toInf_TuneCP5_13TeV_pythia8/RunIISummer20UL16NanoAODv9-106X_mcRun2_asymptotic_v17-v1/NANOAODSIM', '/QCD_Pt_2400to3200_TuneCP5_13TeV_pythia8/RunIISummer20UL16NanoAODv9-106X_mcRun2_asymptoti

In [7]:
fname = "coffeaOutput/trijetHists_wXSscaling_QCDsim_pt200.0rapidity2.5_bbloosejesjecALL.pkl"
with open(fname, "rb") as f:
    result = pickle.load( f )
print(result.keys())
response_matrix_u_values, ptreco_edges, mreco_edges, ptgen_edges, mgen_edges = result['response_matrix_u'][{'syst':syst}].project("ptreco", "mreco", "ptgen", "mgen").to_numpy(flow=True)
print(response_matrix_g_values.shape)
response_matrix_g_values= result['response_matrix_g'][{'syst':'nominal'}].project("ptreco", "mreco", "ptgen", "mgen").values()
nptreco,nmassreco,nptgen,nmassgen = response_matrix_g_values.shape
response_matrix_g_final = response_matrix_g_values.reshape( (nptreco)*(nmassreco), (nptgen)*(nmassgen) ) 
response_matrix_u_final = response_matrix_u_values.reshape( (nptreco+2)*(nmassreco+2), (nptgen+2)*(nmassgen+2) ) 
print(response_matrix_g_final.shape)
response_matrix_g_final2 = result['response_matrix_g'][{'syst':"nominal"}].project("ptreco", "mreco", "ptgen", "mgen").values(flow=True).reshape( (nptreco+2)*(nmassreco+2), (nptgen+2)*(nmassgen+2))
print(np.sum(response_matrix_g_final[26,:]))
print(np.sum(response_matrix_g_final2[26,:]))
print(np.sum(response_matrix_u_final[26,:]))

dict_keys(['jet_mass', 'jet_pt', 'jet_eta', 'btag', 'cutflow', 'njet_gen', 'dphimin_gen', 'asymm_gen', 'njet_reco', 'dphimin_reco', 'asymm_reco', 'jet_dr_reco_gen', 'jet_mass_reco_over_gen', 'jet_pt_reco', 'jet_pt_gen', 'jet_pt_reco_over_gen', 'jet_eta_reco', 'jet_eta_gen', 'jet_rap_reco', 'jet_rap_gen', 'jet_dphi_reco', 'jet_ptasymm_reco', 'jet_dr_gen_subjet', 'jet_dr_reco_to_gen_subjet', 'jet_sd_mass_reco', 'jet_sd_mass_gen', 'misses_g', 'fakes_g', 'misses', 'fakes', 'jet_pt_mass_reco_u', 'jet_pt_mass_reco_g', 'jet_pt_mass_gen_u', 'jet_pt_mass_gen_g', 'jet_m_pt_u_reco_over_gen', 'jet_m_pt_g_reco_over_gen', 'response_matrix_u', 'response_matrix_g', 'weights', 'systematics'])
(12, 26, 12, 14)
(240, 120)
1750.0
763.0
5.596834033787367


In [8]:
# fname = uproot.open('root://cmsxrootd.fnal.gov//store/mc/RunIISummer20UL18NanoAODv9/QCD_Pt_1400to1800_TuneCP5_13TeV_pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v1/280000/42B4DEA0-C0E7-A944-80DC-1D107A0F63FB.root')

In [9]:
# print(fname["Events"].keys("*HTXS*"))

In [10]:
hists = uproot.open('rootFiles/trijetHistsQCDsim_jecALL.root')
print(hists.keys())
hist_vals_g, hist_edges, hist_widths = hists['ptreco_mreco_g_nominal'].to_numpy(flow=True)
hist_vals_u, hist_edges, hist_widths = hists['ptreco_mreco_u_nominal'].to_numpy(flow=True)
print(hist_vals_g[1][0])
print(hist_vals_u[1][0])

['ptgen_mgen_u_nominal;1', 'ptgen_mgen_g_nominal;1', 'ptreco_mreco_u_nominal;1', 'ptreco_mreco_g_nominal;1', 'misses_ptgen_mgen_nominal;1', 'fakes_ptreco_mreco_nominal;1', 'ptgen_mgen_u_jesUp;1', 'ptgen_mgen_g_jesUp;1', 'ptreco_mreco_u_jesUp;1', 'ptreco_mreco_g_jesUp;1', 'misses_ptgen_mgen_jesUp;1', 'fakes_ptreco_mreco_jesUp;1', 'ptgen_mgen_u_jesDown;1', 'ptgen_mgen_g_jesDown;1', 'ptreco_mreco_u_jesDown;1', 'ptreco_mreco_g_jesDown;1', 'misses_ptgen_mgen_jesDown;1', 'fakes_ptreco_mreco_jesDown;1', 'ptgen_mgen_u_jerUp;1', 'ptgen_mgen_g_jerUp;1', 'ptreco_mreco_u_jerUp;1', 'ptreco_mreco_g_jerUp;1', 'misses_ptgen_mgen_jerUp;1', 'fakes_ptreco_mreco_jerUp;1', 'ptgen_mgen_u_jerDown;1', 'ptgen_mgen_g_jerDown;1', 'ptreco_mreco_u_jerDown;1', 'ptreco_mreco_g_jerDown;1', 'misses_ptgen_mgen_jerDown;1', 'fakes_ptreco_mreco_jerDown;1', 'response;1']
2112.0
6.716195120548779


In [11]:
hists = uproot.open('rootFiles/trijetHistsQCDsim_jecALL.root')
print(hists['response'].keys())
resp_groomed = hists['response']['groomed_nominal'].array().to_numpy()[0].reshape((nptreco+2)*(nmassreco+2), (nptgen+2)*(nmassgen+2))
print(np.sum(resp_groomed[26,:]))

['groomed_nominal', 'ungroomed_nominal', 'groomed_jesUp', 'ungroomed_jesUp', 'groomed_jesDown', 'ungroomed_jesDown', 'groomed_jerUp', 'ungroomed_jerUp', 'groomed_jerDown', 'ungroomed_jerDown']
2112.0
